In [1]:
# coding=utf-8
import re
import pymorphy2
import pandas as pd
import json
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
from matplotlib import rc
import matplotlib

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [10]:
train_set = pd.read_csv("train_set.csv")
channels = ['big_data', 'datasets', 'deep_learning', 'lang_python', 'lang_r', 'mltrainings_beginners', 'nlp', 'sequences_series', 'theory_and_practice', 'visualization', 'career', '_random_flood']
train_set = train_set[train_set['channel'].isin(channels)]
train_set = train_set[(train_set['timestamp'] < '2017-01-01') & (train_set['timestamp'] > '2016-01-01')]
train_set = train_set.loc[(train_set["reactions"] != '{}')]

users = pd.read_json('data/users.json')
users = users.rename(columns={'id': 'user_id'})
train_set = pd.merge(train_set, users[['user_id','real_name']], on='user_id')

In [11]:
troll_reactions = ['kekeke', 'povar', 'ban', 'trollface_peeking', 'youknow', 'joy', 'laughing', 'slow']

In [12]:
def extract_count_troll(x):
    x = x.replace("'", "\"")
    x = json.loads(x)
    if x["name"] in troll_reactions:
        return x["count"]
    return 0
train_set["reactions_count"] = train_set["reactions"].apply(lambda x: extract_count_troll(x))
trolls = train_set

In [13]:
count = trolls.groupby(["channel", "real_name",])["reactions_count"].sum().reset_index()

In [15]:
overall_trolls_2016 = count.sort_values(["reactions_count"], ascending=False)
#overall_trolls_2016[1:10]
overall_trolls_2016 = overall_trolls_2016.to_csv('overall_trolls_2016.csv', sep='\t', encoding='utf-8')